In [3]:
import numpy as np
import pandas as pd 
import os 
from astropy import units as u
from astropy.cosmology import FlatLambdaCDM
parent_dir=os.path.dirname(os.getcwd())

def read_line_emission_csv(path_line_emission_csv):
    """
    Read the csv file in which are stored the line emission's rest frequency.
    
    Parameter: 
    path_line_emission_csv (str): Path to file.csv within there are the line emission's rest frequency.
    
    Return:
    pd.DataFrame : Dataframe with line names and rest frequencies.
    """
    db_line = pd.read_csv(path_line_emission_csv, sep = ";")
    return db_line

def sed_reading(type, path):
    cosmo = FlatLambdaCDM(H0=70 * u.km / u.s / u.Mpc, Tcmb0=2.725 * u.K, Om0=0.3)
    if type == "extended":
        file_path = path + "/sed_low_z_warm_star_forming_galaxy.dat"
    elif type == "point":
        file_path = path + "/sed_low_z_type2_AGN.dat"
    else:
        return "Not valid type"
    
    try: 
        SED= pd.read_csv(file_path, sep="\s+")
        rename_columns = {
            'um' : 'GHz',
            'erg/s/Hz': 'Jy',
        }
        SED.rename(columns=rename_columns, inplace=True)
        SED['GHz']=SED['GHz'].apply(lambda x: (x* u.um).to(u.GHz, equivalencies=u.spectral()).value)
        SED['Jy']=SED['Jy']/((10.**(-26.))*(10.**7.)*4.*np.pi*(cosmo.luminosity_distance(0.0000000000000000000000000001).value*(3.086e+22))**2.)
        return SED
    except FileNotFoundError:
         return "File not Found"

def process_spectral_data(redshift, central_frequency, delta_freq, source_frequency, lines=None, n_lines=None):
    """
    Process spectral data based on the type of source, wavelength conversion,
    line ratios, and given frequency bands.

    :param redshift: Redshift value to adjust the spectral lines and continuum.
    :param central_frequency: Central frequency of the observation band (GHz).
    :param delta_freq: Bandwidth around the central frequency (GHz).
    :param source_frequency: Frequency of the source obtained from metadata (GHz).
    :param lines: Optional list of line names provided by the user.
    :param n_lines: Number of additional lines to consider if lines is None.
    """
    # Define the frequency range based on central frequency and bandwidth
    freq_min = central_frequency - delta_freq / 2
    freq_max = central_frequency + delta_freq / 2

    # Example data: Placeholder for continuum and lines from SED processing
    SED=sed_reading('point',os.path.join(parent_dir,'brightnes'))

    # Placeholder for line data: line_name, observed_frequency (GHz), line_ratio, line_error
    db_line = read_line_emission_csv(os.path.join(parent_dir,'brightnes','Calibrations_FIR(GHz).csv'))
    
    # Shift the continuum and line frequencies by (1 + redshift)
    SED['GHz'] *= (1 + redshift)
    db_line['freq(GHz)'] = db_line['freq(GHz)'] * (1 + redshift)

    # Filter the continuum and lines to only include those within the frequency range
    continuum_mask = (continuum_frequencies >= freq_min) & (continuum_frequencies <= freq_max)
    filtered_continuum_frequencies = continuum_frequencies[continuum_mask]
    filtered_continuum_values = continuum_values[continuum_mask]

    line_mask = (spectral_lines[:, 1].astype(float) >= freq_min) & (spectral_lines[:, 1].astype(float) <= freq_max)
    filtered_lines = spectral_lines[line_mask]

    # Determine which lines to output based on user input or nearest lines to source frequency
    if lines is not None:
        # Filter out the lines that are not possible
        possible_lines = filtered_lines[np.isin(filtered_lines[:, 0], lines)]
        if len(possible_lines) == 0:
            print("Warning: No valid lines provided by the user. Returning possible lines.")
            possible_lines = filtered_lines
    else:
        # If no lines are provided, select the nearest line to the source frequency and additional lines if specified
        distances = np.abs(filtered_lines[:, 1].astype(float) - source_frequency)
        nearest_index = np.argmin(distances)
        if n_lines is None or n_lines > len(filtered_lines):
            n_lines = len(filtered_lines)
        nearest_lines = filtered_lines[np.argsort(distances)[:n_lines]]
        possible_lines = nearest_lines

    # Output the processed arrays and line information
    return filtered_continuum_frequencies, filtered_continuum_values, possible_lines[:, 0], possible_lines

# Example call to the function with hypothetical parameters
continuum_freqs, continuum_vals, line_names, line_info = process_spectral_data(
    redshift=0.1, 
    central_frequency=5, 
    delta_freq=1, 
    source_frequency=4.85
)

print("Filtered Continuum Frequencies:", continuum_freqs)
print("Filtered Continuum Values:", continuum_vals)
print("Line Names:", line_names)
print("Line Information:", line_info)


ValueError: attempt to get argmin of an empty sequence